In [20]:
from datetime import datetime as dt
from datetime import timedelta
from edgar import Company
import json
import lxml.html as lh
import numpy as np
import pandas as pd
import pandas_datareader as pdr
pd.options.display.max_rows = 100
import re
import requests
import time

In [2]:
# Functions for getting the CIK and Company Name from a SPAC ticker

def get_ticker_to_cik():
    # local copy: data/ticker_to_cik.txt
    ticker_to_cik = pd.read_csv('https://www.sec.gov/include/ticker.txt',
                                sep='\t', header=None, names=['ticker','cik'])
    ticker_to_cik['ticker'] = ticker_to_cik.ticker.str.upper()
    ticker_to_cik['cik'] = ticker_to_cik.cik.astype(str)
    return ticker_to_cik

def get_cik_to_name():
    # local copy: data/cik_to_name.json
    cik_to_name = pd.read_json('https://www.sec.gov/files/company_tickers.json').transpose()
    cik_to_name['ticker'] = cik_to_name.ticker.str.upper()
    cik_to_name['cik'] = cik_to_name.cik_str.astype(str)
    return cik_to_name

def process_spac_lists(file_path_current, file_path_past, write=False):
    # current spac list
    spac_list_current = pd.read_csv(file_path_current)
    spac_list_current = spac_list_current.Ticker.unique()
    spac_list_current = pd.DataFrame(spac_list_current, columns=['Ticker'])
    
    # past spac list (completed business combination)
    spac_list_past = pd.read_csv(file_path_past)
    spac_list_past.fillna('missing', inplace=True)
    spac_list_past['dupe_filter'] = spac_list_past['Old Ticker'] + spac_list_past['New Ticker']
    spac_list_past = spac_list_past[spac_list_past.dupe_filter.isin(spac_list_past.dupe_filter.unique())]
    spac_list_past.drop(columns=['dupe_filter'], inplace=True)
    
    # write to file
    if write==True:
        spac_list_current.to_csv('spac_list_current.csv', index=False)
        spac_list_past.to_csv('spac_list_past.csv', index=False)
    
    # get ticker to cik and cik to company name file, then merge
    ticker_to_cik = get_ticker_to_cik()
    cik_to_name = get_cik_to_name()
    spac_list_past = spac_list_past.merge(ticker_to_cik, how='left', left_on='New Ticker', right_on='ticker')
    spac_list_past = spac_list_past.merge(cik_to_name[['cik','ticker','title']], how='left', on=['cik','ticker'])
    spac_list_current = spac_list_current.merge(ticker_to_cik, how='left', left_on='Ticker', right_on='ticker')
    spac_list_current = spac_list_current.merge(cik_to_name[['cik','ticker','title']], how='left', on=['cik','ticker'])
    
    return spac_list_current, spac_list_past

In [3]:
# Functions for pulling 8-K html, converting to text, grabbing date, and simple classifier

def form_html_to_text(forms_html):
    forms_text = []
    for form_html in forms_html:
        form_text = form_html.text_content().replace('\n',' ').replace('\xa0',' ').lower()
        form_text = re.sub(' +', ' ', form_text) # remove extra spaces
        forms_text.append(form_text)
    return forms_text

def date_parser_primary(split_text):
    # handle case: "november 13 (november 8, 2019)""
    if '(' in split_text[1] and ',' not in split_text[1].split('(')[0]:
        monthday_split = split_text[1].split('(')[0]
        year_split = split_text[1].split('(')[1].split(', ')[1].split(')')[0]
        date_string = monthday_split.replace(' ','') + ', ' + year_split.replace(')','')
    # handle cases:
    # "november 13, 2019"
    # "november 13, 2019 (november 15, 2019)"...uses first date
    # "july [   ], 2019"...return None
    else:
        split_text = split_text[1].split(', ')
        if '[' in split_text[0]:
            return None
        date_string = split_text[0].replace(' ','') + ', ' + split_text[1].replace(' ','')[0:4]
    date_dt = dt.strptime(date_string, '%B%d, %Y')
    date_dt = date_dt.strftime('%Y-%m-%d')
    return date_dt

def date_parser_secondary(split_text):
    # handle case when date comes before "date of report..." string
    split_text = split_text[0].split(', ')
    monthday_split = split_text[len(split_text)-2].split(' ')
    month = monthday_split[len(monthday_split)-2].replace('1934','').replace('(','') # case: 1934June
    day = monthday_split[len(monthday_split)-1]
    year = split_text[len(split_text)-1].replace(' ','')[0:4]
    date_string = str(month)+str(day)+', '+str(year)
    date_dt = dt.strptime(date_string, '%B%d, %Y')
    date_dt = date_dt.strftime('%Y-%m-%d')
    return date_dt

# TODO: handle duplicate dates (sometime multiple 8-Ks on same date)
def create_date_text_df(forms_text, form_type):
    df = pd.DataFrame()
    for form_text in forms_text:
        date_case_1 = 'date of report (date of earliest event reported):'
        date_case_2 = 'date of report (date earliest event reported):'
        date_case_3 = 'date of earliest event reported:'
        date_case_4 = 'date of report (date of earliest event reported)'
        if len(form_text.split(date_case_1))>1:
            split_text = form_text.split(date_case_1)
            date_dt = date_parser_primary(split_text)
        elif len(form_text.split(date_case_2))>1:
            split_text = form_text.split(date_case_2)
            date_dt = date_parser_primary(split_text)
        elif len(form_text.split(date_case_3))>1:
            split_text = form_text.split(date_case_3)
            date_dt = date_parser_primary(split_text)
        elif len(form_text.split(date_case_4))>1:
            split_text = form_text.split(date_case_4)
            date_dt = date_parser_secondary(split_text)
        else:
            print('none of the date identifiers worked. skipping this form...')
#             print(form_text)
            continue
        df = df.append(pd.Series([date_dt, form_type, form_text]), ignore_index=True)
    if len(df)!=0:
        df.columns = ['date','form','text']
    return df

def get_forms_text(company_name, cik_id, form_type):
    print(company_name)
    company = Company(company_name, cik_id)
    print('url to forms:', company.get_filings_url(filing_type=form_type, ownership='include', no_of_entries=100))
    forms_site_html = company.get_all_filings(filing_type=form_type, ownership='include', no_of_entries=100)
    forms_html = company.get_documents(forms_site_html, no_of_documents=100, debug=False)
    forms_text = form_html_to_text(forms_html)
    if len(forms_text)==0:
        return
    df = create_date_text_df(forms_text, form_type)
    return df

def simple_text_match(df_form, substring):
    df_form[substring.replace(' ','_')+'_found'] = df_form.text.apply(lambda x: 1 if substring in x else 0)
    return df_form

In [4]:
# Functions for grabbing and saving price data

def bulk_save_prices_to_csv(symbols, start_date, end_date, source, is_warrant=False):
    for symbol in symbols:
        print(symbol)
        df_prices = get_historical_prices(symbol=symbol,
                                          start_date=start_date,
                                          end_date=end_date,
                                          source=source,
                                          is_warrant=is_warrant)
        if df_prices is not None:
            df_prices = process_historical_prices(df_prices)
            df_prices.to_csv('data/prices_'+source+'/'+symbol+'_prices.csv', index=False)
        time.sleep(5)
        
def run_bulk_save_prices(spac_list_current, spac_list_past, source, is_warrant=False):
    # spacs missing price data entirely
    missing_past_spacs = ['missing', 'LCAH', 'FMCI1', 'CFCO']
    missing_current_spacs = ['ACNDU', 'ARYB', 'BRLI', 'DFHT', 'DMYD', 'FUSE', 'GOAC', 'IWAC', 'LCAH', 'LGVW',
                             'MCAC', 'MLAC', 'PANA', 'PSAC', 'PSTH', 'SSMC', 'TREB']
    
    symbols_past_new_ticker = [x for x in spac_list_past.ticker.unique().tolist() if str(x)!='nan']
    symbols_past_old_ticker = spac_list_past['Old Ticker'].unique().tolist()
    symbols_past_old_ticker = [x for x in symbols_past_old_ticker if x not in missing_past_spacs]
    symbols_current = [x for x in spac_list_current['Ticker'] if x not in missing_current_spacs]
    if is_warrant:
        symbols_past_new_ticker = [x + 'W' for x in symbols_past_new_ticker]
        symbols_past_old_ticker = [x + 'W' for x in symbols_past_old_ticker]
        symbols_current = [x + 'W' for x in symbols_current]
    bulk_save_prices_to_csv(symbols=symbols_past_new_ticker, start_date='2018-01-01',
                            end_date=dt.today().strftime('%Y-%m-%d'), source=source, is_warrant=is_warrant)
    bulk_save_prices_to_csv(symbols=symbols_past_old_ticker, start_date='2018-01-01',
                            end_date=dt.today().strftime('%Y-%m-%d'), source=source, is_warrant=is_warrant)
    bulk_save_prices_to_csv(symbols=symbols_current, start_date='2018-01-01',
                            end_date=dt.today().strftime('%Y-%m-%d'), source=source, is_warrant=is_warrant)
        
def load_saved_prices_data(symbol):
    df_prices = pd.read_csv('data/prices/'+symbol+'_prices.csv')
    print('price data min date:', df_prices.date.min())
    print('price data max date:', df_prices.date.max())
    return df_prices

def millis_since_epoch_to_dt(x):
    return dt.fromtimestamp(float(x)/1000).strftime('%Y-%m-%d %H:%M:%S.%f')

def dt_to_millis_since_epoch(x):
    epoch = dt.utcfromtimestamp(0)
    return str(int((x-epoch).total_seconds()*1000))

def get_daily_prices_td(symbol, start_date, end_date):
    TD_CONSUMER_KEY = 'NSAF00ABB0EREWY1A0GGUSDGIPGQQOA1'
    start_date_epoch = dt_to_millis_since_epoch(dt.strptime(start_date, '%Y-%m-%d'))
    end_date_epoch = dt_to_millis_since_epoch(dt.strptime(end_date, '%Y-%m-%d'))
    endpoint = 'https://api.tdameritrade.com/v1/marketdata/{symbol}/pricehistory?periodType=month&frequencyType=daily&startDate={start_date}&endDate={end_date}'
    # endpoint for minute-to-minute prices (only a couple months available):
    # endpoint = 'https://api.tdameritrade.com/v1/marketdata/{symbol}/pricehistory?periodType=day&frequencyType=minute&startDate={start_date}&endDate={end_date}'
    full_url = endpoint.format(symbol=symbol,
                               start_date=start_date_epoch,
                               end_date=end_date_epoch)
    page = requests.get(url=full_url, params={'apikey' : TD_CONSUMER_KEY})
    content = json.loads(page.content)
    try:
        df_td_prices = pd.DataFrame.from_dict(content['candles'])
    except:
        return None
    if len(df_td_prices)==0:
        return None
    df_td_prices['datetime'] = df_td_prices.datetime.apply(lambda x: millis_since_epoch_to_dt(x))
    df_td_prices['date'] = df_td_prices.datetime.apply(lambda x: x[0:10])
    df_td_prices['ticker'] = symbol
    return df_td_prices

def get_historical_prices(symbol, start_date, end_date, source, is_warrant=False):
    print('getting prices from', start_date, 'to', end_date)
    if source=='td':
        df_prices = get_daily_prices_td(symbol, start_date, end_date)
        if df_prices is None and is_warrant:
            df_prices = get_daily_prices_td(symbol[:-1]+'.WS', start_date, end_date)
    else:
        start_split = start_date.split('-')
        end_split = end_date.split('-')
        start = dt(int(start_split[0]), int(start_split[1]), int(start_split[2]))
        end = dt(int(end_split[0]), int(end_split[1]), int(end_split[2]))
        try:
            # be careful with missing/limited data in yahoo data
            if source=='yahoo':
                df_prices = pdr.data.DataReader(name=symbol, data_source='yahoo', start=start, end=end)
            # alphavantage seems to be most reliable. 5 calls per minute and 500 calls per day
            if source=='alphavantage':
                df_prices = pdr.data.DataReader(name=symbol, data_source='av-daily', start=start, end=end,
                                                api_key='BDB9WJQRCZKINCLD')
            # iex has extremely limited api calls
            if source=='iex':
                df_prices = pdr.data.DataReader(name=symbol, data_source='iex', start=start, end=end,
                                                api_key='pk_970dfff359894b15a056cf677c02e11f')
            df_prices.reset_index(inplace=True)
            df_prices.rename(columns={'index':'date'}, inplace=True)
        except:
            return None
    return df_prices

def process_historical_prices(df_prices):
    df_prices.columns = df_prices.columns.str.lower()
    df_prices['date'] = df_prices.date.astype(str)
    df_prices['close_t+1'] = df_prices.close.shift(-1)
    df_prices['close_t+3'] = df_prices.close.shift(-3)
    df_prices['close_t+5'] = df_prices.close.shift(-5)
    df_prices['close_t+7'] = df_prices.close.shift(-7)
    df_prices['open_close_t+1_%chg'] = (df_prices['close_t+1'] - df_prices['open']) / df_prices['open']
    df_prices['open_close_t+3_%chg'] = (df_prices['close_t+3'] - df_prices['open']) / df_prices['open']
    df_prices['open_close_t+5_%chg'] = (df_prices['close_t+5'] - df_prices['open']) / df_prices['open']
    df_prices['open_close_t+7_%chg'] = (df_prices['close_t+7'] - df_prices['open']) / df_prices['open']
    df_prices = df_prices.round(2)
    print('output min date:', df_prices.date.min())
    print('output max date:', df_prices.date.max())
    return df_prices

In [16]:
# Function outputting dataframe of 8-Ks, classification, and 1,3,5,7 day returns

def agg_spac_returns(spac_list, price_source='alphavantage', is_warrant=False):
    df_returns_agg = pd.DataFrame()
    for ind in range(0, len(spac_list)):
        row = spac_list.iloc[ind]
        print('index:', ind)
        print(row.ticker)
        
        broken_current_spacs = ['GNRS','KBLM','LGC','LIVE','NOVS']
        if is_warrant:
            broken_current_spacs = [x + 'W' for x in broken_current_spacs]
        if row.ticker in broken_current_spacs:
            print('in broken spac list, skipping...')
            continue

        # get form 8Ks
        df_form_8K = get_forms_text(company_name=row.title, cik_id=row.cik, form_type='8-K')
        if df_form_8K is None or len(df_form_8K)==0:
            print('no 8Ks found\n')
            continue
            
        # simple classifier
        df_form_8K = simple_text_match(df_form_8K, 'letter of intent')
        df_form_8K = simple_text_match(df_form_8K, 'business combination agreement')

        # get stock prices
#         try:
#             df_prices = load_saved_prices_data(row.ticker)
#         except:
#             df_prices = get_historical_prices(symbol=row.ticker,
#                                               start_date='2018-01-01',
#                                               end_date=dt.today().strftime('%Y-%m-%d'),
#                                               source=price_source,
#                                               is_warrant=is_warrant)
#             if df_prices is None:
#                 print('prices for', row.ticker, 'not found. skipping...')
#                 continue
#             df_prices = process_historical_prices(df_prices)
        df_prices = get_historical_prices(symbol=row.ticker,
                                          start_date='2018-01-01',
                                          end_date=dt.today().strftime('%Y-%m-%d'),
                                          source=price_source,
                                          is_warrant=is_warrant)
        if df_prices is None:
            print('prices for', row.ticker, 'not found. skipping...\n')
            continue
        df_prices = process_historical_prices(df_prices)

        # output returns per form
        df_returns = df_form_8K.merge(df_prices[['date','open_close_t+1_%chg','open_close_t+3_%chg',
                                                 'open_close_t+5_%chg','open_close_t+7_%chg']],
                                      how='left', on='date')
        display(df_returns)

        # aggregate
        if len(df_returns)!=0:
            df_returns['symbol'] = row.ticker
        df_returns_agg = df_returns_agg.append(df_returns)
        
        # sec site sometimes will timeout
        if ind%30==0 and ind!=0:
            time.sleep(120)
        
    # drop duplicates on date + symbol
    df_returns_agg = df_returns_agg.drop_duplicates(subset=['date','symbol'])
    
    return df_returns_agg

In [6]:
# load current and past spac lists
spac_list_current, spac_list_past = process_spac_lists(file_path_current='data/spac_list_current.csv',
                                                       file_path_past='data/spac_list_past.csv',
                                                       write=False)

# bulk save price data for symbols in spac lists (due to API limits)...no need to run if already saved
# run_bulk_save_prices(spac_list_current, spac_list_past, source='td', is_warrant=False)
# run_bulk_save_prices(spac_list_current, spac_list_past, source='td', is_warrant=True)

In [17]:
# get returns following 8-Ks for past spacs
# df_returns_past = agg_spac_returns(spac_list_past, price_source='td')

# get returns following 8-Ks for current spacs
# df_returns_current = agg_spac_returns(spac_list_current, price_source='td')

# get returns following 8-Ks for current spac warrants
spac_list_current_warrants = spac_list_current.copy()
spac_list_current_warrants['ticker'] = spac_list_current_warrants.Ticker + 'W' # warrants
df_returns_current_warrants = agg_spac_returns(spac_list_current_warrants,
                                               price_source='td',
                                               is_warrant=True)

index: 0
ACAMW
Acamar Partners Acquisition Corp.
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1759008&type=8-K&dateb=&owner=include&count=100
getting prices from 2018-01-01 to 2020-07-15
output min date: 2019-04-15
output max date: 2020-07-14


,date,form,text,letter_of_intent_found,business_combination_agreement_found,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
0,2020-07-13,8-K,8-k 1 tm2024803d1_8k.htm form 8-k united stat...,0,0,-0.07,NaN,NaN,NaN
1,2020-02-20,8-K,8-k 1 tm209532d1_8k.htm form 8-k united state...,0,0,NaN,NaN,NaN,NaN
2,2019-04-12,8-K,8-k 1 tv518793_8k.htm form 8-k united states ...,0,0,NaN,NaN,NaN,NaN
3,2019-04-10,8-K,8-k 1 tv518539_8k.htm form 8-k united states ...,0,0,NaN,NaN,NaN,NaN
4,2019-03-04,8-K,8-k 1 tv515277_8k.htm form 8-k united states ...,0,0,NaN,NaN,NaN,NaN
5,2019-02-26,8-K,8-k 1 tv514540_8k.htm form 8-k united states ...,0,0,NaN,NaN,NaN,NaN


index: 1
ACTTW
Whole Earth Brands, Inc.
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1753706&type=8-K&dateb=&owner=include&count=100
getting prices from 2018-01-01 to 2020-07-15
output min date: 2019-05-14
output max date: 2020-06-24


,date,form,text,letter_of_intent_found,business_combination_agreement_found,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
0,2020-06-24,8-K,8-k12b/a 1 tm2023499-2_8k12ba.htm 8-k12b/a un...,0,0,NaN,NaN,NaN,NaN
1,2020-06-24,8-K,8-k12b 1 tm2023499d1_8k.htm form 8-k12b unite...,0,0,NaN,NaN,NaN,NaN
2,2020-06-15,8-K,8-k 1 tm2022707d1_8k.htm form 8-k united stat...,0,0,0.12,0.54,0.07,0.27
3,2020-06-15,8-K,8-k 1 tm2022614d1_8k.htm form 8-k united stat...,0,0,0.12,0.54,0.07,0.27
4,2020-06-08,8-K,8-k 1 tm2022016d1_8k.htm form 8-k united stat...,0,0,0.06,-0.03,0.30,0.54
5,2020-05-08,8-K,8-k 1 tm2019220d1_8k.htm form 8-k united stat...,0,0,NaN,NaN,NaN,NaN
6,2020-03-27,8-K,8-k 1 ea120112actii_8k.htm united states secu...,0,0,NaN,NaN,NaN,NaN
7,2020-02-12,8-K,8-k 1 tv538156_8k.htm 8-k united states secur...,0,0,NaN,NaN,NaN,NaN
8,2020-01-10,8-K,8-k 1 tv536223_8k.htm form 8-k united states ...,0,0,0.04,0.21,0.21,0.31
9,2019-12-19,8-K,8-k 1 tv535070_8k.htm form 8-k united states ...,0,0,NaN,NaN,NaN,NaN


index: 2
ALACW
Alberton Acquisition Corp
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1748621&type=8-K&dateb=&owner=include&count=100
getting prices from 2018-01-01 to 2020-07-15
output min date: 2018-11-21
output max date: 2020-07-14


...

index: 121
LPROW
Open Lending Corp
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1806201&type=8-K&dateb=&owner=include&count=100
getting prices from 2018-01-01 to 2020-07-15
prices for LPROW not found. skipping...

index: 122
MCACW
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found

index: 123
MLACW
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found

index: 124
NBACW
Newborn Acquisition Corp
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1780262&type=8-K&dateb=&owner=include&count=100
getting prices from 2018-01-01 to 2020-07-15
output min date: 2020-04-30
output max date: 2020-07-14


,date,form,text,letter_of_intent_found,business_combination_agreement_found,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
0,2020-04-29,8-K,8-k 1 ea121166-8k_newbornacq.htm current repo...,0,0,NaN,NaN,NaN,NaN
1,2020-02-19,8-K,8-k 1 f8k021920b_newbornacq.htm current repor...,0,0,NaN,NaN,NaN,NaN
2,2020-02-13,8-K,8-k 1 f8k021320_newbornacq.htm current report...,0,0,NaN,NaN,NaN,NaN


index: 125
NOVSW
in broken spac list, skipping...
index: 126
PANAW
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found

index: 127
PSACW
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found

index: 128
PSTHW
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found

index: 129
SSMCW
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found

index: 130
THBRW
Thunder Bridge Acquisition II, LTD
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1769318&type=8-K&dateb=&owner=include&count=100
getting prices from 2018-01-01 to 2020-07-15
output min date: 2019-10-04
output max date: 2020-07-14


,date,form,text,letter_of_intent_found,business_combination_agreement_found,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
0,2019-10-02,8-K,8-k 1 f8k100219_thunderbridge2.htm from 8-k u...,0,0,NaN,NaN,NaN,NaN
1,2019-08-19,8-K,8-k 1 f8k081319_thunderbridge2.htm current re...,0,0,NaN,NaN,NaN,NaN
2,2019-08-13,8-K,8-k 1 f8k080819_thunderbridge2.htm form 8k un...,0,0,NaN,NaN,NaN,NaN


index: 131
TREBW
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found

index: 132
WPFW
nan
url to forms: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=nan&type=8-K&dateb=&owner=include&count=100
no 8Ks found



## Analysis

In [27]:
# Functions for backtest
# note .123 is 12.3% in output

def compute_mean_returns(df_returns, corrupt_symbols):
    df_mean_returns = df_returns[~df_returns.symbol.isin(corrupt_symbols)].groupby('symbol').agg(
        {'open_close_t+1_%chg':'mean','open_close_t+3_%chg':'mean',
         'open_close_t+5_%chg':'mean','open_close_t+7_%chg':'mean'})
    df_mean_returns.dropna(inplace=True)
    print('Count:', len(df_mean_returns))
    return np.round(df_mean_returns.sort_values(by='open_close_t+3_%chg'), 3)

def summary_statistics(df_returns, corrupt_symbols):
    return np.round(df_returns[~df_returns.symbol.isin(corrupt_symbols)].dropna().describe(), 3)

def compute_mean_returns_LOI(df_returns, corrupt_symbols):
    return np.round(df_returns[(~df_returns.symbol.isin(corrupt_symbols)) &
                               (df_returns.letter_of_intent_found==1)].dropna().groupby('symbol').agg(
        {'open_close_t+1_%chg':'mean','open_close_t+3_%chg':'mean',
         'open_close_t+5_%chg':'mean','open_close_t+7_%chg':'mean'}), 3)


Past SPACs

In [377]:
print('mean returns for past spacs')
compute_mean_returns(df_returns_past, corrupt_symbols=['ACEL'])

In [378]:
print('summary stats for returns in past spacs')
summary_statistics(df_returns_past, corrupt_symbols=['ACEL'])

In [379]:
print('mean returns for past spacs (LOI text match)')
compute_mean_returns_LOI(df_returns_past, corrupt_symbols=['ACEL'])

Current SPACs

In [380]:
print('mean returns for current spacs')
compute_mean_returns(df_returns_current, corrupt_symbols=[])

In [381]:
print('summary stats for returns in current spacs')
summary_statistics(df_returns_current, corrupt_symbols=[])

In [382]:
print('mean returns for current spacs (LOI text match)')
compute_mean_returns_LOI(df_returns_current, corrupt_symbols=[])

Current SPAC Warrants

In [28]:
print('mean returns for current spac warrants')
compute_mean_returns(df_returns_current_warrants, corrupt_symbols=[])

mean returns for current spac warrants
Count: 43


,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
symbol,,,,
CRSAW,-0.430,-0.460,-0.550,-0.540
SAMAW,-0.070,-0.200,0.047,0.113
INSUW,-0.113,-0.080,-0.025,0.130
LOACW,-0.060,-0.060,-0.194,-0.128
JIHW,-0.020,-0.060,0.050,0.020
LHCW,-0.039,-0.043,-0.011,-0.027
CCXXW,-0.210,-0.040,-0.010,0.010
LACQW,0.004,-0.034,-0.124,-0.124
LIVKW,-0.190,-0.030,-0.180,-0.290


In [29]:
print('summary stats for returns in current spac warrants')
summary_statistics(df_returns_current_warrants, corrupt_symbols=[])

summary stats for returns in current spac warrants


,letter_of_intent_found,business_combination_agreement_found,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
count,148.000,148.000,148.000,148.000,148.000,148.000
mean,0.081,0.176,0.105,0.151,0.184,0.269
std,0.274,0.382,0.466,0.559,0.687,1.016
min,0.000,0.000,-0.510,-0.540,-0.730,-0.560
25%,0.000,0.000,-0.050,-0.080,-0.100,-0.095
50%,0.000,0.000,0.000,0.000,0.015,0.060
75%,0.000,0.000,0.092,0.200,0.250,0.315
max,1.000,1.000,3.520,3.760,4.800,7.800


In [30]:
print('mean returns for current spac warrants (LOI text match)')
compute_mean_returns_LOI(df_returns_current_warrants, corrupt_symbols=[])

mean returns for current spac warrants (LOI text match)


,open_close_t+1_%chg,open_close_t+3_%chg,open_close_t+5_%chg,open_close_t+7_%chg
symbol,,,,
ALACW,0.295,0.275,0.450,0.325
AMCIW,-0.180,0.650,0.650,0.880
BMRGW,1.100,1.440,1.000,1.220
FMCIW,1.820,2.120,2.795,2.495
MFACW,0.430,0.120,-0.100,0.610
NFINW,0.330,0.030,0.300,0.380
OPESW,3.180,3.130,4.180,7.000
PAACW,-0.035,-0.070,0.030,-0.005
SAMAW,0.280,0.200,0.820,0.720
